# 🧬 Demo: Genetic Algorithm Individual Class

This notebook demonstrates the functionality of the `Individual` abstract base class and a concrete subclass `BinaryIndividual`. The goal is to show how this flexible design supports mutation, crossover, fitness evaluation, and operator overloading for use in genetic algorithms.

## 🧱 Class Overview

- `Individual`: An abstract class that defines a genetic algorithm individual.
  - Includes pluggable methods: `random_representation`, `mutation`, `crossover`, `calculate_fitness`, and `semantic_key`.
  - Overloads key operators for convenience:
    - `@` → Crossover (e.g., `child = parent1 @ parent2`)
    - `**` → Multiple self-crossovers (e.g., `clones = parent ** 3`)
    - `()` → Evaluate fitness (e.g., `fitness = individual()`)
    - Comparison (`<`, `>`, etc.) based on fitness
    - Hashing and equality via `semantic_key` for use in sets and dictionaries

- `BinaryIndividual`: A concrete implementation where the genome is a list of 0s and 1s.
  - Fitness is defined as the number of 1s (maximize ones).
  - Mutation flips a single bit.
  - Crossover combines parts of the genome from two parents.

## ✅ Features Demonstrated

1. **Random Initialization**
   - Create individuals with random binary genomes.

2. **Fitness Evaluation**
   - Automatically lazily computed and cached.

3. **Mutation**
   - Use `mutation()` to modify an individual in-place.

4. **Crossover**
   - Use the `@` operator (or `crossover()`) to generate a child from two parents.

5. **Iteration**
   - Use a `for` loop to generate several mutated versions from an individual.

6. **Operator Overloading**
   - Use Python syntax sugar for fitness, crossover, and comparisons.

7. **Sorting Individuals**
   - Individuals are comparable by fitness, so they can be sorted.

8. **Using Sets**
   - `Individual` instances are hashable via `semantic_key`, allowing deduplication in sets or as dictionary keys.

This modular and extensible framework allows for easy implementation of different genome types and evolutionary strategies.

In [ ]:
from abc import ABC, abstractmethod
from copy import deepcopy
import random

In [ ]:
# import a script in another directory
import sys
import os
#relative_path = os.path.join("..", "Classes")
print(sys.path)
#sorted(os.listdir(relative_pathhgggggfffddwwddssddbbbbbbeeeeeeeaaaaaaaaaatttttttrrrrrrriiiiiizzzzzzzzzzzzz))


In [ ]:
#Individual.py
from abc import ABC, abstractmethod
from copy import deepcopy


class Individual(ABC):

    def __init__(self):
        self._mutation_probability = 0.5
        self._crossover_probability = 0.5
        self.mutation_count = 5
        self._fitness = None

    @property
    def mutation_probability(self):
        return self._mutation_probability

    @mutation_probability.setter
    def mutation_probability(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("Mutation probability must be an int or float.")
        if not (0 <= value <= 1):
            raise ValueError("Mutation probability must be between 0 and 1.")
        self._mutation_probability = value

    @property
    def crossover_probability(self):
        return self._crossover_probability

    @crossover_probability.setter
    def crossover_probability(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("Crossover probability must be an int or float.")
        if not (0 <= value <= 1):
            raise ValueError("Crossover probability must be between 0 and 1.")
        self._crossover_probability = value

    @abstractmethod
    def random_representation(self) -> None:
        """ Generate a random representation of the individual."""
        pass
    
    @abstractmethod
    def check_representation(self) -> bool:
        """ Check if the representation is valid."""
        return True
    @property
    def fitness(self):
        if self._fitness is None:
            self._fitness = self.calculate_fitness()  # Only calculate when needed but once it is calculated it is stored
        return self._fitness
    
    @abstractmethod
    def calculate_fitness(self):
        pass
    
    # Method for mutation
    @abstractmethod
    def mutation(self):
        pass

    # Method for crossover
    @abstractmethod
    def crossover(self, other):
        pass
    
    
    
    @abstractmethod
    def semantic_key(self):
        """Return a hashable object that defines semantic equivalence.
        This is used to check if two individuals are semantically equivalent.
        In Grouping problems for example, the semantic key is the ordered set of the elements in the genome.

        Returns:
            hashable: A hashable object that defines semantic equivalence.
        
        """
        pass

    def __hash__(self):
        return hash(self.semantic_key())    


    def __iter__(self):
        for _ in range(self.mutation_count):
            yield next(self)
        
    def __next__(self):
        copy = self.copy_Individual(delete_fitness=True)
        copy.mutation()
        return copy


    def __matmul__(self, other):
        return self.crossover(other)
    
    def __pow__(self, power):
         return [self.crossover(self) for _ in range(power)]

    def __call__(self):
        return self.fitness

    def __float__(self):
        return float(self.fitness) if self.fitness is not None else float('-inf')

    # Comparison Methods
    def __eq__(self, other):
        if not isinstance(other, Individual):
            return False
        return self.semantic_key() == other.semantic_key()

    def __ne__(self, other):
        return not self.__eq__(other)

    def __lt__(self, other):
        return float(self) < float(other)
    
    def __le__(self, other):
        return float(self) <= float(other)
    
    def __gt__(self, other):
        return float(self) > float(other)
    
    def __ge__(self, other):
        return float(self) >= float(other)
    
    
    def __len__(self):
        """ Return the length of the genome. """
        return len(self.genome)

    # String Representation
    def __str__(self):
        return f"Fitness: {self.fitness}"
    
    # Deep Copy
    def copy_Individual(self, delete_fitness=True):
        """
        Returns a deep copy of the individual, optionally deleting the fitness attribute.
        This is useful for creating a new individual that is a copy of the current one and are ready to be mutated or crossed over.
        The fitness of the new individual will be set to None if delete_fitness is True.
        This is necessary because the fitness of the new individual will be calculated again after mutation or crossover.
        
        Args:
            delete_fitness (bool): If True, the fitness of the new individual will be set to None.
        Returns:
            Individual: A deep copy of the individual.
        
        """
        new_individual = deepcopy(self)
        if delete_fitness:
            new_individual._fitness = None
        return new_individual


In [ ]:
import os
import pandas as pd
os.chdir(os.pardir) # comment if you are running this more than once
data_matrix_df = pd.read_csv('data/distance_matrix_official.csv', index_col=0)
data_matrix_np = data_matrix_df.to_numpy()
data_matrix_np.shape

In [ ]:
from Genetic_algorithm.fitness import ResourceFitness
from Genetic_algorithm.genome import Genome

MyFitness = ResourceFitness(data_matrix_np)

In [ ]:
my_genome = Genome()

In [ ]:
from copy import deepcopy
from Genetic_algorithm.mutations import logistic_mutation, social_mutation
import random

class BinaryIndividual(Individual):
    def __init__(self, genome_class, fitness_instance):
        super().__init__()
        self.genome_class = genome_class
        self.random_representation()
        self.fitness_instance = fitness_instance

    def random_representation(self):
        self.genome = self.genome_class()

    def check_representation(self):
        return True

    def calculate_fitness(self):
    
        return self.fitness_instance.evaluate(self.genome)

    def mutation(self):
        new_genome = None
        if random.random() < self.prob_social_mutation:
            new_genome =  social_mutation(self.genome)
            
        else:
            new_genome  = logistic_mutation(self.genome)
        
        new_individual = self.copy_Individual(delete_fitness=True)
        new_individual.genome = new_genome
        return new_individual
    

    def crossover(self, other):
        return deepcopy(self),deepcopy(other)

    def semantic_key(self):
        return self.genome.semantic_key()

In [ ]:
'''class BinaryIndividual(Individual):
    def __init__(self, genome_length=10):
        super().__init__()
        self.genome_length = genome_length
        self.genome = [0] * genome_length
        self.random_representation()

    def random_representation(self):
        self.genome = [random.choice([0, 1]) for _ in range(self.genome_length)]

    def check_representation(self):
        return all(g in [0, 1] for g in self.genome)

    def calculate_fitness(self):
        return sum(self.genome)  # Maximize number of 1s

    def mutation(self):
        idx = random.randint(0, self.genome_length - 1)
        self.genome[idx] = 1 - self.genome[idx]  # Flip bit
        return self

    def crossover(self, other):
        pivot = random.randint(1, self.genome_length - 1)
        child = self.copy_Individual()
        child.genome = self.genome[:pivot] + other.genome[pivot:]
        return child

    def semantic_key(self):
        return tuple(self.genome)'''

# Testing Features

## Create two individuals


In [ ]:
ind1 = BinaryIndividual(Genome, MyFitness)
ind2 = BinaryIndividual(Genome, MyFitness)

print("Individual 1:", ind1.genome)
print("Fitness 1:", float(ind1))

print("Individual 2:", ind2.genome)
print("Fitness 2:", float(ind2))

## Crossover


In [ ]:
child1, child2 = ind1 @ ind2
print("Child (crossover):", child1.genome)
print("Fitness (child):", float(child1))

## Mutation


In [ ]:
mutant = ind1.copy_Individual()
mutant.mutation()
print("\nMutated Individual:", mutant.genome)
print("Fitness (mutated):", float(mutant))

## Iteration (generate mutated versions)

In [ ]:
print("\nMultiple Mutations:")
for i, m in enumerate(ind1):
    print(f"Mutation {i+1}:", m.genome, "Fitness:", float(m))

## Operator Overloading

In [ ]:
print("\nUsing ** Operator for Self-Crossover")
clones = ind1 ** 3
for i, (clone1, clone2) in enumerate(clones):
    print(f"Clone pair {i+1}: Lengths = {len(clone1)}, {len(clone2)}")

## Comparison


In [ ]:
print("\nIs Individual 1 better than Individual 2?", ind1 > ind2)

## Sorting and Deduplication via Set

In [ ]:
print("\n=== Sorting Individuals by Fitness ===")
individuals = [BinaryIndividual() for _ in range(5)]

for i, ind in enumerate(individuals):
    print(f"Ind {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")

sorted_individuals = sorted(individuals)
print("\nSorted by fitness (ascending):")
for i, ind in enumerate(sorted_individuals):
    print(f"Rank {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")

## Storing Unique Individuals in a Set

In [ ]:
print("\n=== Deduplication with Set ===")
ind1 = BinaryIndividual()
ind2 = ind1.copy_Individual(delete_fitness=False)  # Exact copy

# Mutate a third individual to likely get a different genome
ind3 = BinaryIndividual()
ind3.mutation()

ind_set = {ind1, ind2, ind3}

for i, ind in enumerate(ind_set):
    print(f"Unique {i+1}: Genome = {ind.genome}, Fitness = {float(ind)}")

print(f"\nTotal unique individuals: {len(ind_set)}")